<a href="https://colab.research.google.com/github/roshniraoof/Twitter-Data-Ingestion/blob/master/Coronavirus%20webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datetime import date
today= date.today()

d= today.strftime("%m-%d-%y")
print("date =", d)

date = 03-16-20


In [0]:
cnn_url="https://edition.cnn.com/world/live-news/coronavirus-outbreak-{}-intl-hnk/index.html".format(d)

In [0]:
from bs4 import BeautifulSoup
import requests

In [0]:
html = requests.get(cnn_url).text

In [10]:
soup = BeautifulSoup(html)
print(soup.title)

<title data-rh="true">Coronavirus news and live updates: First participant in vaccine trial given dose   - CNN</title>


In [0]:
import spacy

nlp = spacy.load('en_core_web_sm')

In [0]:
for link in soup.find_all("h2"):
  print("Headline : {} ".format(link.text))
  for ent in nlp(link.text).ents:
    print ("\tText : {} , Entity : {} ".format(ent.text,ent.label_))

In [0]:
nbc_url = 'https://www.nbcnews.com/health/coronavirus'
cnbc_rss_url = 'https://www.cnbc.com/id/10000108/device/rss/rss.html'

In [0]:
urls = [cnn_url, nbc_url, cnbc_rss_url ]
formats = ['html.parser', 'html.parser', 'xml']
tags = ['h2', 'h2', 'description']
website = ['CNN', 'NBC', 'CNBC']


In [0]:
crawl_len = 0
for url in urls:
   print("Crawling Webpage.................{}".format(url))
   response = requests.get(url)
   soup = BeautifulSoup(response.content,formats[crawl_len])

   for link in soup.find_all(tags[crawl_len]):
     if (len(link.text.split(" ")) > 4):
       print("Headline : {}".format(link.text))

       entities = []
       for ent in nlp(link.text).ents:
         print ("\tText : {} , Entity : {} ".format(ent.text,ent.label_))
   crawl_len=crawl_len+1



In [0]:
crawl_len = 0
news_dict =[]
for url in urls:
   response = requests.get(url)
   soup = BeautifulSoup(response.content,formats[crawl_len])

   for link in soup.find_all(tags[crawl_len]):
     if (len(link.text.split(" ")) > 4):
       

       entities = []
       entities = [(ent.text,ent.label_) for ent in nlp(link.text).ents]
       news_dict.append({'website': website[crawl_len], 'url': url , 'headline':link.text ,'entities' : entities} )
   crawl_len=crawl_len+1




In [25]:
news_dict

[{'entities': [],
  'headline': 'What you need to know now',
  'url': 'https://edition.cnn.com/world/live-news/coronavirus-outbreak-03-16-20-intl-hnk/index.html',
  'website': 'CNN'},
 {'entities': [('Georgia', 'GPE'), ('Wednesday', 'DATE')],
  'headline': 'Georgia will close all public schools starting Wednesday',
  'url': 'https://edition.cnn.com/world/live-news/coronavirus-outbreak-03-16-20-intl-hnk/index.html',
  'website': 'CNN'},
 {'entities': [('Ohio', 'GPE'), ('June', 'DATE')],
  'headline': 'Ohio judge allows state to move presidential primary to June ',
  'url': 'https://edition.cnn.com/world/live-news/coronavirus-outbreak-03-16-20-intl-hnk/index.html',
  'website': 'CNN'},
 {'entities': [('NASCAR', 'ORG'), ('May 3', 'DATE')],
  'headline': 'NASCAR postpones all races through May 3',
  'url': 'https://edition.cnn.com/world/live-news/coronavirus-outbreak-03-16-20-intl-hnk/index.html',
  'website': 'CNN'},
 {'entities': [('Los Angeles County Sheriff', 'GPE')],
  'headline': 'Lo

In [0]:
import pandas as pd
news_df = pd.DataFrame(news_dict)

In [28]:
pd.set_option('max_colwidth', 800)
news_df.head(20)

,website,url,headline,entities
0,CNN,https://edition.cnn.com/world/live-news/coronavirus-outbreak-03-16-20-intl-hnk/index.html,What you need to know now,[]
1,CNN,https://edition.cnn.com/world/live-news/coronavirus-outbreak-03-16-20-intl-hnk/index.html,Georgia will close all public schools starting Wednesday,"[(Georgia, GPE), (Wednesday, DATE)]"
2,CNN,https://edition.cnn.com/world/live-news/coronavirus-outbreak-03-16-20-intl-hnk/index.html,Ohio judge allows state to move presidential primary to June,"[(Ohio, GPE), (June, DATE)]"
3,CNN,https://edition.cnn.com/world/live-news/coronavirus-outbreak-03-16-20-intl-hnk/index.html,NASCAR postpones all races through May 3,"[(NASCAR, ORG), (May 3, DATE)]"
4,CNN,https://edition.cnn.com/world/live-news/coronavirus-outbreak-03-16-20-intl-hnk/index.html,Los Angeles County Sheriff releases inmates early due to coronavirus concerns,"[(Los Angeles County Sheriff, GPE)]"
5,CNN,https://edition.cnn.com/world/live-news/coronavirus-outbreak-03-16-20-intl-hnk/index.html,"""Saturday Night Live"" suspends production until further notice","[(Saturday Night Live, WORK_OF_ART)]"
6,CNN,https://edition.cnn.com/world/live-news/coronavirus-outbreak-03-16-20-intl-hnk/index.html,"New limits on UK, Ireland travel poses another test for Trump administration after weekend airport meltdown","[(UK, GPE), (Ireland, GPE), (Trump, LOC), (weekend, DATE)]"
7,CNN,https://edition.cnn.com/world/live-news/coronavirus-outbreak-03-16-20-intl-hnk/index.html,"All schools in New York ordered to close by Wednesday, governor says","[(New York, GPE), (Wednesday, DATE)]"
8,CNN,https://edition.cnn.com/world/live-news/coronavirus-outbreak-03-16-20-intl-hnk/index.html,McDonald's USA will close seating areas in restaurants due to outbreak,"[(McDonald's USA, ORG)]"
9,CNN,https://edition.cnn.com/world/live-news/coronavirus-outbreak-03-16-20-intl-hnk/index.html,Bars to close and restaurants services to be limited in Kentucky,"[(Kentucky, GPE)]"


In [29]:
pd.concat([news_df[['website','url', 'headline',]], news_df['entities'].apply(pd.Series)],axis=1)

,website,url,headline,0,1,2,3,4,5,6
0,CNN,https://edition.cnn.com/world/live-news/coronavirus-outbreak-03-16-20-intl-hnk/index.html,What you need to know now,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CNN,https://edition.cnn.com/world/live-news/coronavirus-outbreak-03-16-20-intl-hnk/index.html,Georgia will close all public schools starting Wednesday,"(Georgia, GPE)","(Wednesday, DATE)",NaN,NaN,NaN,NaN,NaN
2,CNN,https://edition.cnn.com/world/live-news/coronavirus-outbreak-03-16-20-intl-hnk/index.html,Ohio judge allows state to move presidential primary to June,"(Ohio, GPE)","(June, DATE)",NaN,NaN,NaN,NaN,NaN
3,CNN,https://edition.cnn.com/world/live-news/coronavirus-outbreak-03-16-20-intl-hnk/index.html,NASCAR postpones all races through May 3,"(NASCAR, ORG)","(May 3, DATE)",NaN,NaN,NaN,NaN,NaN
4,CNN,https://edition.cnn.com/world/live-news/coronavirus-outbreak-03-16-20-intl-hnk/index.html,Los Angeles County Sheriff releases inmates early due to coronavirus concerns,"(Los Angeles County Sheriff, GPE)",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
101,CNBC,https://www.cnbc.com/id/10000108/device/rss/rss.html,European markets slid on Monday as much of the region went into shutdown mode to prevent the spread of the new coronavirus.,"(European, NORP)","(Monday, DATE)",NaN,NaN,NaN,NaN,NaN
102,CNBC,https://www.cnbc.com/id/10000108/device/rss/rss.html,"Susie Buffett sat next to someone who had just returned from a trip to Spain and subsequently rested positive for COVID-19, the Omaha World-Herald reported.","(Susie Buffett, PERSON)","(Spain, GPE)","(the Omaha World-Herald, ORG)",NaN,NaN,NaN,NaN
103,CNBC,https://www.cnbc.com/id/10000108/device/rss/rss.html,"Night clubs, movie theaters, small theater houses and concert venues will also close, according to de Blasio.","(de Blasio, PERSON)",NaN,NaN,NaN,NaN,NaN,NaN
104,CNBC,https://www.cnbc.com/id/10000108/device/rss/rss.html,"Germany is the latest European country to seal off its borders in an effort to contain the coronavirus outbreak, as the number of deaths in Europe jumped overnight.","(Germany, GPE)","(European, NORP)","(Europe, LOC)",NaN,NaN,NaN,NaN
